In [ ]:
# neural network
import torch
from torch import nn
from torchvision import models, transforms

from collections import OrderedDict

from PIL import Image

import helper

import os

In [ ]:
# path
PATH = os.getcwd()
PATH

In [ ]:
# load model
state_dict = torch.load(PATH + '/transfer_trained_model/cat_dog_checkpoint.pth', map_location=torch.device('cpu'))
#print(state_dict.keys())

In [ ]:
# model architecture
model = models.resnet18(pretrained=True)

# freeze parameters so we do not backpropagate through them
for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 256)), 
    ('relu', nn.ReLU()), 
    ('dropout', nn.Dropout(0.2)), 
    ('fc2', nn.Linear(256, 2)), 
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = classifier

#model

In [ ]:
# load state dict
model.load_state_dict(torch.load(PATH + '/transfer_trained_model/cat_dog_checkpoint.pth', map_location=torch.device('cpu')))
model.eval()

In [ ]:
# transform
test_transform = transforms.Compose([
    transforms.Resize(255), 
    transforms.CenterCrop(224), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], 
                        [0.229, 0.224, 0.225])
])

# load image
basewidth = 300

img = Image.open(PATH + '/katito_karla/01.jpg')

wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img.resize((basewidth,hsize), Image.ANTIALIAS)

In [ ]:
img = test_transform(img)
img.unsqueeze_(0)

# predictions
logps = model.forward(img)
ps = torch.exp(logps)

In [ ]:
# Plot the image and probabilities
helper.view_classify(img, ps, version='CatDog')